In [9]:
from awpy import Demo

import torch
from torch_geometric_temporal.signal import DynamicHeteroGraphTemporalSignal

import pandas as pd
import polars as pl
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from termcolor import colored
import time
import json
import pickle
import sys
import os

pd.set_option('display.max_columns', 100)
pd.set_option('future.no_silent_downcasting', True)

pl.Config.set_tbl_rows(10)

sys.path.append(os.path.abspath('../../package'))

from CS2.graph import TabularGraphSnapshot, HeteroGraphSnapshot, TemporalHeteroGraphSnapshot
from CS2.token import Tokenizer
from CS2.preprocess import Dictionary, NormalizePosition, NormalizeTabularGraphSnapshot, ImputeTabularGraphSnapshot
from CS2.visualize import HeteroGraphVisualizer

DATA_PATH = '../../data/matches-processed/cs2/hetero-graph/'
DATA_SAVE_PATH = '../../data/matches-processed/cs2/temporal-hetero-graph/'
PROCESS_SAVE_PATH = './parses/temp-hetero-parse-2024.10.10/'

# Create Temporal dataset

In [26]:
hetero_graph_matches = [f for f in os.listdir(DATA_PATH) if os.path.isfile(os.path.join(DATA_PATH, f)) and f not in ['train.pt', 'val.pt']]
processed_matches = []

dataset_lengths = []
overall_length = 0

# Load processed matches
if os.path.exists(os.path.join(PROCESS_SAVE_PATH, 'process.txt')):
    with open(os.path.join(PROCESS_SAVE_PATH, 'process.txt'), 'r') as f:
        processed_matches = f.readlines()

# Remove whitespace characters like `\n` at the end of each line
for match_idx in range(len(processed_matches)):
    processed_matches[match_idx] = processed_matches[match_idx].strip()


for file in hetero_graph_matches:

    if file in processed_matches:
        print(colored(f'{file} already processed. Skipping...', 'yellow'))
        continue

    print(colored(f'Processing {file}...', 'light_blue'))

    match = torch.load(DATA_PATH + file, weights_only=False)
    thgs = TemporalHeteroGraphSnapshot()
    dyn_graphs = None
    dyn_graphs = thgs.process_match(match, interval=20, shifted_intervals=True)

    dataset_lengths.append(len(dyn_graphs))
    overall_length += len(dyn_graphs)

    print('DTDG Dataset Length:', len(dyn_graphs))
    print(colored(f'{file} processed.', 'green'))

    torch.save(dyn_graphs, DATA_SAVE_PATH + file)

    with open(os.path.join(PROCESS_SAVE_PATH, 'process.txt'), 'a') as f:
        f.write(f'{file}\n')

Processing 100000.pt...
DTDG Dataset Length: 708
100000.pt processed.
Processing 100001.pt...
DTDG Dataset Length: 792
100001.pt processed.
Processing 100002.pt...
DTDG Dataset Length: 990
100002.pt processed.
Processing 100003.pt...
DTDG Dataset Length: 761
100003.pt processed.
Processing 100004.pt...
Error:Error: There are missing ticks in the graph sequence. The error occured while parsing match 100004.0 at round                                 0.1666666716337204 between ticks 35158.0-35463.0. Skipping the sequence.
DTDG Dataset Length: 875
100004.pt processed.
Processing 100005.pt...
DTDG Dataset Length: 862
100005.pt processed.
Processing 100006.pt...
DTDG Dataset Length: 804
100006.pt processed.
Processing 100007.pt...
DTDG Dataset Length: 756
100007.pt processed.
Processing 100008.pt...
DTDG Dataset Length: 1061
100008.pt processed.
Processing 100009.pt...
DTDG Dataset Length: 890
100009.pt processed.
Processing 100010.pt...
Error:Error: There are missing ticks in the graph sequ

In [27]:
print(overall_length)

98098
